In [17]:
#References:
#https://stackoverflow.com/questions/185936/how-to-delete-the-contents-of-a-folder-in-python
#http://www.markhneedham.com/blog/2015/02/15/pythonscikit-learn-calculating-tfidf-on-how-i-met-your-mother-transcripts/
    
from bs4 import BeautifulSoup
import requests
import re

kick_response = requests.get("https://webrobots.io/kickstarter-datasets/")
t = kick_response.text
soup = BeautifulSoup(t, 'html.parser')

link = re.findall('https://s3.amazonaws.com/weruns/forfun/Kickstarter.*\.zip', t)

zipfiles = []
for x in link:
    if x.find('JSON')!= 139:
        zipfiles.append(x[117:208])

#Picking up one zip file for running the analysis
zipurl = zipfiles[1]
zipurl

'https://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2017-10-15T10_20_38_271Z.zip'

In [18]:
import requests, zipfile, io
import glob
import pandas as pd
import os, shutil

# Create own path
path ='tmp'

#Combine all unzipped csv into one pandas dataframe
df = pd.DataFrame()
lst = []

#This loop can be used for extracting files from all the URLs

req = requests.get(zipurl)
zip_ = zipfile.ZipFile(io.BytesIO(req.content))
zip_.extractall("tmp") #note this folder needs to be created on the root 

Files = glob.glob(path + "/*.csv")
for file in Files:
    df = pd.read_csv(file,index_col=None, header=0)
    lst.append(df)

#Deleting unzipped files
for the_file in os.listdir(path):
    file_path = os.path.join(path, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
    except Exception as e:
        print(e)

df = pd.concat(lst)

In [ ]:
# For concatenating data in all the 25 zips

#import requests, zipfile, io
#import glob
#import pandas as pd
#import os, shutil

# Create own path
#path ='tmp/stuff'


#Combine all unzipped csv into one dataframe
#df = pd.DataFrame()
#lst = []

#This loop can be used for extracting files from all the URLs (obviously the indentation needs to be handled)
#for zipurl in zipfiles: 
#    req = requests.get(zipurl)
#    zip_ = zipfile.ZipFile(io.BytesIO(req.content))
#    zip_.extractall("tmp/stuff")

#    Files = glob.glob(path + "/*.csv")
#    for file in Files:
#        df = pd.read_csv(file,index_col=None, header=0)
#        lst.append(df)

    #Deleting unzipped files
#    for the_file in os.listdir(path):
#        file_path = os.path.join(path, the_file)
#        try:
#            if os.path.isfile(file_path):
#                os.unlink(file_path)
#        except Exception as e:
#            print(e)

#df = pd.concat(lst)

In [19]:
#Standardizing/cleaning the data before analysis
regex = re.compile('[^a-zA-Z ]')
regex2= re.compile('\s+')

df['name_new'] = df['name'].apply(lambda x: regex.sub('', str(x).lower()))
df['name_new'] = df['name_new'].apply(lambda x: regex2.sub(' ', str(x).lower()))

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Specifying the options; analysing two word phrases
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0, stop_words = 'english')

tfidf_matrix =  tf.fit_transform(df['name_new'])
feature_names = tf.get_feature_names() 

#FYI: kickstarter has initiative data
dens = tfidf_matrix.todense()
initiative = dens[0].tolist()[0]
phrase_scores = [pair for pair in zip(range(0, len(initiative)), initiative) if pair[1] > 0]

#Display the weights/scores of the two word phrases
sort_phrase_scores = sorted(phrase_scores, key=lambda t: t[1] * -1)
for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sort_phrase_scores][:15]:
   print('{0: <15} {1}'.format(phrase, score))
 

fantasy music   0.314881503330133
fantasy music video 0.314881503330133
melinehs        0.314881503330133
melinehs rock   0.314881503330133
melinehs rock roll 0.314881503330133
roll fantasy music 0.314881503330133
rock roll fantasy 0.30447758318984997
roll fantasy    0.30447758318984997
rock roll       0.23333510737331936
roll            0.21136601805975733
rock            0.17842266974379709
fantasy         0.16789327637420143
music video     0.1587579683542306
video           0.14088482495731233
music           0.12445216816242695
